In [2]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
import urllib
from IPython.display import clear_output

In [3]:
db_url = "mysql+mysqlconnector://{USER}:{PWD}@{HOST}/{DBNAME}"

db_url = db_url.format(
    USER = "root", 
    PWD = "QsLdzoV7u9ZxuIeL",
    HOST = "localhost:3306",
    DBNAME = "eventfrogdata"
)

In [4]:
sql_query = """
SELECT 
    `web-scraper-order`,
    name,
    description,
    price,
    address,
    category,
    time,
    MAKEDATE(2023, 334 + SUBSTRING(date, 5, 3)) AS date_formatted
FROM
    events_raw
WHERE
    is_eventgruppe IS NULL
ORDER BY date_formatted ASC;
"""

In [5]:
engine = create_engine(db_url, echo=False)

with engine.begin() as conn:
    
    df_events = pd.read_sql_query(
        sql=sql_query, # query
        con=conn # database connection 
    )

In [6]:
print(df_events.shape)
df_events.head()

(1671, 8)


,web-scraper-order,name,description,price,address,category,time,date_formatted
0,1701178304-1235,Live-Musik mit Sina Anastasia,Ihr Name *ein Hauch von Melodie*\n Ihre Ausstr...,None,"Kellertheater im Haus der Vereine, Baselstrass...",Konzert / Singer & Songwriter,19:00 Uhr\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\...,2023-12-01
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",Metal Marmot im Dezember 2023: Freitag 1. Deze...,CHF 40.00,"Hotel Simplon, Kanderstegstrasse 25, Frutigen, CH",Konzert / Hard Rock & Heavy Metal,Beginn:\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t1...,2023-12-01
2,1701178308-1237,JAMfreeday,Alle sind herzlich eingeladen um zu musizieren...,None,"Dä 3.Stock, Industriestrasse, 28, Herisau, CH",Konzert,19:00 Uhr,2023-12-01
3,1701178310-1238,Am Hummelwald,"Der Kurzprosaband ""Am Hummelwald - Miniaturen ...",None,"Raum für Literatur, St.Leonhard-Strasse 40, St...",Konzert,19:00 Uhr,2023-12-01
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,BELPHEGOR MARSCHIERT DURCH EUROPA WINTER MMXXI...,CHF 38.00,"Musigburg, Aarburg, CH",Konzert / Hard Rock & Heavy Metal,19:00 Uhr\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\...,2023-12-01


In [7]:
df_events['price'] = df_events['price'].str.extract('(\d+.\d+)').astype(float)
df_events['price']

0        NaN
1       40.0
2        NaN
3        NaN
4       38.0
        ... 
1666     NaN
1667     NaN
1668     NaN
1669     NaN
1670     NaN
Name: price, Length: 1671, dtype: float64

In [8]:
df_events._get_value(1, 'time')

'Beginn:\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t19:00 Uhr'

In [9]:
df_time = df_events['time'].str.extractall('(\d+:\d+)').unstack()

In [10]:
df_events['time_from'] = df_time[0][0]
df_events['time_until'] = df_time[0][1]
df_events = df_events.drop(columns='time')
df_events.head()

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until
0,1701178304-1235,Live-Musik mit Sina Anastasia,Ihr Name *ein Hauch von Melodie*\n Ihre Ausstr...,NaN,"Kellertheater im Haus der Vereine, Baselstrass...",Konzert / Singer & Songwriter,2023-12-01,19:00,22:00
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",Metal Marmot im Dezember 2023: Freitag 1. Deze...,40.0,"Hotel Simplon, Kanderstegstrasse 25, Frutigen, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,NaN
2,1701178308-1237,JAMfreeday,Alle sind herzlich eingeladen um zu musizieren...,NaN,"Dä 3.Stock, Industriestrasse, 28, Herisau, CH",Konzert,2023-12-01,19:00,NaN
3,1701178310-1238,Am Hummelwald,"Der Kurzprosaband ""Am Hummelwald - Miniaturen ...",NaN,"Raum für Literatur, St.Leonhard-Strasse 40, St...",Konzert,2023-12-01,19:00,NaN
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,BELPHEGOR MARSCHIERT DURCH EUROPA WINTER MMXXI...,38.0,"Musigburg, Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,23:50


In [11]:
df_events['duration_seconds'] = pd.to_datetime(df_events['time_until'], format='%H:%M') - pd.to_datetime(df_events['time_from'], format='%H:%M')
df_events['duration_seconds'] = df_events['duration_seconds'].dt.seconds

In [12]:
df_events['is_concert'] = df_events['category'].str.contains('Konzert').astype(int)

In [13]:
df_events['description'] = df_events['description'].replace(r'\n',' ', regex=True)
df_events['description'] = df_events['description'].str.lower()
df_events['description_len'] = df_events['description'].str.len()

In [14]:
df_events.head()

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until,duration_seconds,is_concert,description_len
0,1701178304-1235,Live-Musik mit Sina Anastasia,ihr name *ein hauch von melodie* ihre ausstra...,NaN,"Kellertheater im Haus der Vereine, Baselstrass...",Konzert / Singer & Songwriter,2023-12-01,19:00,22:00,10800.0,1,2860.0
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",metal marmot im dezember 2023: freitag 1. deze...,40.0,"Hotel Simplon, Kanderstegstrasse 25, Frutigen, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,NaN,NaN,1,245.0
2,1701178308-1237,JAMfreeday,alle sind herzlich eingeladen um zu musizieren...,NaN,"Dä 3.Stock, Industriestrasse, 28, Herisau, CH",Konzert,2023-12-01,19:00,NaN,NaN,1,188.0
3,1701178310-1238,Am Hummelwald,"der kurzprosaband ""am hummelwald - miniaturen ...",NaN,"Raum für Literatur, St.Leonhard-Strasse 40, St...",Konzert,2023-12-01,19:00,NaN,NaN,1,1060.0
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,belphegor marschiert durch europa winter mmxxi...,38.0,"Musigburg, Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,23:50,17400.0,1,734.0


### Delete the house name in the address if there is one

In [15]:
df_adr = df_events["address"].str.rsplit(",", n=3, expand=True)
df_adr['address_prepared'] = df_adr[1] + ', ' + df_adr[2]
df_adr

,0,1,2,3,address_prepared
0,Kellertheater im Haus der Vereine,Baselstrasse 43,Riehen,CH,"Baselstrasse 43, Riehen"
1,Hotel Simplon,Kanderstegstrasse 25,Frutigen,CH,"Kanderstegstrasse 25, Frutigen"
2,"Dä 3.Stock, Industriestrasse",28,Herisau,CH,"28, Herisau"
3,Raum für Literatur,St.Leonhard-Strasse 40,St.Gallen,CH,"St.Leonhard-Strasse 40, St.Gallen"
4,Musigburg,Aarburg,CH,None,"Aarburg, CH"
...,...,...,...,...,...
1666,Werk 1,Fabrikstr. 7,Gossau SG,CH,"Fabrikstr. 7, Gossau SG"
1667,Luftseilbahn Jakobsbad-Kronberg AG,St. Josefstrasse 2,Jakobsbad,CH,"St. Josefstrasse 2, Jakobsbad"
1668,Luftseilbahn Jakobsbad-Kronberg AG,St. Josefstrasse 2,Jakobsbad,CH,"St. Josefstrasse 2, Jakobsbad"
1669,Kloster Fischingen,Kloster Fischingen,Fischingen,CH,"Kloster Fischingen, Fischingen"


In [16]:
df_events["address"] = df_adr["address_prepared"]
df_events.head()

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until,duration_seconds,is_concert,description_len
0,1701178304-1235,Live-Musik mit Sina Anastasia,ihr name *ein hauch von melodie* ihre ausstra...,NaN,"Baselstrasse 43, Riehen",Konzert / Singer & Songwriter,2023-12-01,19:00,22:00,10800.0,1,2860.0
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",metal marmot im dezember 2023: freitag 1. deze...,40.0,"Kanderstegstrasse 25, Frutigen",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,NaN,NaN,1,245.0
2,1701178308-1237,JAMfreeday,alle sind herzlich eingeladen um zu musizieren...,NaN,"28, Herisau",Konzert,2023-12-01,19:00,NaN,NaN,1,188.0
3,1701178310-1238,Am Hummelwald,"der kurzprosaband ""am hummelwald - miniaturen ...",NaN,"St.Leonhard-Strasse 40, St.Gallen",Konzert,2023-12-01,19:00,NaN,NaN,1,1060.0
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,belphegor marschiert durch europa winter mmxxi...,38.0,"Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,23:50,17400.0,1,734.0


# Geocoding addresses

In [17]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df_events['address'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df_events['address']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        



Geocoding address 1671 out of 1671 :  Churerstrasse 10,  Rorschach


In [18]:
# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df_events['address'])))
df_events['lat'] = df_loc['lat']
df_events['lon'] = df_loc['lon']
df_events.head(5)

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until,duration_seconds,is_concert,description_len,lat,lon
0,1701178304-1235,Live-Musik mit Sina Anastasia,ihr name *ein hauch von melodie* ihre ausstra...,NaN,"Baselstrasse 43, Riehen",Konzert / Singer & Songwriter,2023-12-01,19:00,22:00,10800.0,1,2860.0,47.584694,7.649675
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",metal marmot im dezember 2023: freitag 1. deze...,40.0,"Kanderstegstrasse 25, Frutigen",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,NaN,NaN,1,245.0,46.586052,7.647597
2,1701178308-1237,JAMfreeday,alle sind herzlich eingeladen um zu musizieren...,NaN,"28, Herisau",Konzert,2023-12-01,19:00,NaN,NaN,1,188.0,47.387520,9.309265
3,1701178310-1238,Am Hummelwald,"der kurzprosaband ""am hummelwald - miniaturen ...",NaN,"St.Leonhard-Strasse 40, St.Gallen",Konzert,2023-12-01,19:00,NaN,NaN,1,1060.0,47.422333,9.370071
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,belphegor marschiert durch europa winter mmxxi...,38.0,"Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023-12-01,19:00,23:50,17400.0,1,734.0,47.324833,7.905110


## Export for work in QGIS

In [19]:
df_events.to_csv('data/events_before_qgis.csv')

## Import after data enrichment with QGIS

In [27]:
df_events = pd.read_csv("data/events_after_qgis.csv")

## Translate Kantonsnummer to Kantonsabkürzung

In [28]:
df_kantone = pd.read_csv("data/kantonsnummern.csv", index_col=0)
df_kantone.head()

,kanton
kantonsnum,
1,ZH
2,BE
3,LU
4,UR
5,SZ


In [29]:
df_events = df_events.join(df_kantone, on="kantonsnum")
df_events = df_events.drop(columns="kantonsnum")
df_events.head()

,web-scraper-order,name,description,price,address,category,date_formatted,time_from,time_until,duration_seconds,is_concert,description_len,lat,lon,bfs_nummer,gemeinde,distance_bahnhof,kanton
0,1701178304-1235,Live-Musik mit Sina Anastasia,ihr name *ein hauch von melodie* ihre ausstra...,NaN,"Baselstrasse 43, Riehen",Konzert / Singer & Songwriter,2023/12/01,19:00:00,22:00:00,10800.0,True,2860.0,47.584694,7.649675,2703,Riehen,231.871315,BS
1,1701178306-1236,"Metal Marmot, 1. - 2. Dezember 2023",metal marmot im dezember 2023: freitag 1. deze...,40.0,"Kanderstegstrasse 25, Frutigen",Konzert / Hard Rock & Heavy Metal,2023/12/01,19:00:00,NaN,NaN,True,245.0,46.586052,7.647597,563,Frutigen,431.905393,BE
2,1701178308-1237,JAMfreeday,alle sind herzlich eingeladen um zu musizieren...,NaN,"28, Herisau",Konzert,2023/12/01,19:00:00,NaN,NaN,True,188.0,47.387520,9.309265,3001,Herisau,1931.580777,AR
3,1701178310-1238,Am Hummelwald,"der kurzprosaband ""am hummelwald - miniaturen ...",NaN,"St.Leonhard-Strasse 40, St.Gallen",Konzert,2023/12/01,19:00:00,NaN,NaN,True,1060.0,47.422333,9.370071,3203,St. Gallen,94.997717,SG
4,1701178312-1239,BELPHEGOR 30 Year Tour Special Guest: Arkona &...,belphegor marschiert durch europa winter mmxxi...,38.0,"Aarburg, CH",Konzert / Hard Rock & Heavy Metal,2023/12/01,19:00:00,23:50:00,17400.0,True,734.0,47.324833,7.905110,4271,Aarburg,559.350472,AG


## Export prepared data

In [31]:
df_events.to_csv("data/events_prepared.csv")

### Jupyter notebook --footer info--

In [32]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.1.0
Datetime: 2023-12-02 23:19:22
Python Version: 3.9.13
-----------------------------------
